In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import filters
import pandas as pd




In [ ]:
import os

folder_path = '/content/drive/MyDrive/TestingCarves'

# List all files and subdirectories in the folder
contents = os.listdir(folder_path)

# Print the list of contents
print(contents[0])


In [ ]:
elements=[]
listofarrays=[]
widths=int(input("width please: "))
heights=int(input("height please: "))
elementamount=int(input("amount of elements: "))
for i in range(elementamount):
  elements.append(input("enter the label: "))
elements=np.array(elements)
print(elements)




In [ ]:
for d in range(3):
  test = np.array(Image.open('/content/drive/MyDrive/TestingCarves/'+contents[d]))
  test2=test.copy()
  print(test.shape)
  check=0
  realwidth,realheight,_=test.shape
  scalew=widths/realwidth
  scaleh=heights/realheight
#print(scalew)
#print(scaleh)
  if (scalew<scaleh):
    realscale=scaleh
  else:
    check=1
    realscale=scalew
  realwidth=round(realwidth*realscale)
  realheight=round(realheight*realscale)
  test2=cv2.resize(test2,(realheight,realwidth))
  if (check==1):
    amountchange=realheight-heights
  else:
    test2=cv2.rotate(test2, cv2.ROTATE_90_CLOCKWISE)
    amountchange=realwidth-widths
  for i in range(amountchange):
    test2=seeminglycarvable(test2)
  if (check==0):
    test2=cv2.rotate(test2, cv2.ROTATE_90_COUNTERCLOCKWISE)
  listofarrays.append(test2)
  print(test2.shape)
  plt.imshow(test2)
  plt.show()
  plt.imshow(test)
  plt.show()

In [ ]:
def seeminglycarvable(test2):
  gray = cv2.cvtColor(test2, cv2.COLOR_BGR2GRAY)
  imag = filters.sobel(gray.astype("float"))
  #plt.imshow(imag, cmap='gray')
  #plt.show()
  energy_map = imag
  row, column = energy_map.shape

  cumulative_energy = np.zeros_like(energy_map)
  cumulative_energy[0, :] = energy_map[0, :]
  for r in range(1, row):
    for c in range(0, column):
      if (c==0):
        cumulative_energy[r,c]=energy_map[r,c]+min(cumulative_energy[r-1,c],cumulative_energy[r-1,c+1])
      elif (c==column-1):
        cumulative_energy[r,c]=energy_map[r,c]+min(cumulative_energy[r-1,c-1],cumulative_energy[r-1,c])
      else:
        cumulative_energy[r,c]=energy_map[r,c]+min(cumulative_energy[r-1,c-1],cumulative_energy[r-1,c],cumulative_energy[r-1,c+1])
  energy_maps=cumulative_energy
  row, column = cumulative_energy.shape
  seam=[]

  currentpixel=np.argmin(energy_maps[-1,:])
  for i in reversed(range(row)):
    seam.append(currentpixel)
    if currentpixel == 0:
        nextpixel = np.argmin(energy_maps[i, currentpixel:currentpixel + 2])
    elif currentpixel == column - 1:
        nextpixel = np.argmin(energy_maps[i, currentpixel - 1:currentpixel]) + currentpixel - 1
    else:
        nextpixel = np.argmin(energy_maps[i, currentpixel - 1:currentpixel + 2]) + currentpixel - 1

    currentpixel = nextpixel
  #print(seam)
  #x=0
  #copies=test2.copy()
  #for i in range(row-1,-1,-1):
    #copies[i,seam[x]]=0
    #x=x+1
  #plt.imshow(copies)
  #plt.show()
  seam.reverse()
  r, c, _ = test2.shape
  newImg = np.zeros((r, c, 3))
  for i, j in enumerate(seam):
      newImg[i, 0:j, :] = test2[i, 0:j, :]
      newImg[i, j:c-1, :] = test2[i, j+1:c, :]
  test2=newImg[:, :-1, :].astype(np.uint8)
  #print(test2.shape)
  return test2


In [ ]:
print(listofarrays)
newlistof = listofarrays[1:]

data = {
    'image':newlistof,
    'classification': elements
}
pd.DataFrame(data)
plt.imshow(listofarrays[1])
plt.show()